In [14]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error
import tensorflow as tf

In [15]:
# Check for GPU/TPU
device_name = tf.test.gpu_device_name()
if device_name != '':
    print(f'Found GPU at: {device_name}')
else:
    print('No GPU found, using CPU.')

No GPU found, using CPU.


In [16]:
## loading data
nestdf = pd.read_csv('Nest.csv')
partavgdf = pd.read_csv('Part_Avg.csv')

In [17]:
joined_nest_partsavg_df = pd.merge(nestdf, partavgdf, on='ixJobSummary', how='inner')
joined_nest_partsavg_df #avg num of parts and nest data in one df

,ixJobSummary,fStrategies,dLengthUsed_Avg,dWidthUsed_Avg,dPartArea_Job,dTrueAreaRectified_Job,dLength_Avg,dWidth_Avg,dArea_Avg,cNested_Avg,fExtShape_Avg,dExtArea_Avg,dExtBoundaryDist_Avg,dExtContainedDist_Avg,dLgIntArea_Avg,dLgIntBoundaryDist_Avg,dLgIntContainedDist_Avg,dLgExtConArea_Avg,dLgExtConBoundaryDist,dLgExtConContainedDist
0,130032,0,240.000000,95.990000,519.0404,23037.600000,11.214286,7.588371,58.148629,1,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,145075,8,108.453240,39.263200,18389.9746,21573.573062,38.347309,12.843400,475.076800,4,5,496.283591,6.026536,39.581636,14.593373,0.535145,2.846391,12.247745,0.341673,0.728891
2,233110,8,38.709400,35.736100,879.9165,1383.322989,37.702200,20.209600,439.958200,2,0,439.958200,9.161500,36.520700,0.000000,0.000000,0.000000,60.162600,2.761000,12.675600
3,286531,8,69.701700,60.558550,5992.2342,9448.707993,17.103650,18.125000,191.540050,12,0,193.776600,6.589750,20.581050,0.279550,0.000000,0.125000,35.632250,2.375000,4.500000
4,504531,0,6.807000,38.006900,198.8594,258.712968,37.000000,5.800000,198.859400,1,0,198.859400,2.644400,37.148100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790931,2583832,0,143.770400,94.319500,3562.2303,13560.352243,52.203125,32.046875,815.453875,1,2,1175.660175,15.350675,52.728425,355.015300,7.125000,15.062500,0.000000,0.000000,0.000000
1790932,548877,4,34.513950,55.300350,2898.6813,4526.656120,23.053571,13.199986,273.049857,4,3,296.410943,6.052186,25.126200,3.782143,0.233057,1.326471,0.000000,0.000000,0.000000
1790933,2420784,8,83.969400,33.398500,1732.8873,2804.452006,83.057750,4.540683,288.814567,1,16,331.347183,2.041967,83.173917,10.515283,0.609483,2.738883,0.000000,0.000000,0.000000
1790934,2302130,8,80.844067,54.065133,7208.6328,14136.765442,43.584450,40.175350,693.526650,5,0,1308.887900,15.457300,51.257850,504.934150,7.507200,22.396550,61.498750,1.939500,9.697650


In [18]:
joined_nest_partsavg_df['CropUtil'] = joined_nest_partsavg_df['dPartArea_Job'] / joined_nest_partsavg_df['dTrueAreaRectified_Job']

joined_nest_partsavg_df.drop('dLengthUsed_Avg', axis=1, inplace=True)
joined_nest_partsavg_df.drop('dWidthUsed_Avg', axis=1, inplace=True)
joined_nest_partsavg_df.drop('dTrueAreaRectified_Job', axis=1, inplace=True)


joined_nest_partsavg_df.drop('ixJobSummary', axis=1, inplace=True)

joined_nest_partsavg_df

,fStrategies,dPartArea_Job,dLength_Avg,dWidth_Avg,dArea_Avg,cNested_Avg,fExtShape_Avg,dExtArea_Avg,dExtBoundaryDist_Avg,dExtContainedDist_Avg,dLgIntArea_Avg,dLgIntBoundaryDist_Avg,dLgIntContainedDist_Avg,dLgExtConArea_Avg,dLgExtConBoundaryDist,dLgExtConContainedDist,CropUtil
0,0,519.0404,11.214286,7.588371,58.148629,1,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022530
1,8,18389.9746,38.347309,12.843400,475.076800,4,5,496.283591,6.026536,39.581636,14.593373,0.535145,2.846391,12.247745,0.341673,0.728891,0.852431
2,8,879.9165,37.702200,20.209600,439.958200,2,0,439.958200,9.161500,36.520700,0.000000,0.000000,0.000000,60.162600,2.761000,12.675600,0.636089
3,8,5992.2342,17.103650,18.125000,191.540050,12,0,193.776600,6.589750,20.581050,0.279550,0.000000,0.125000,35.632250,2.375000,4.500000,0.634186
4,0,198.8594,37.000000,5.800000,198.859400,1,0,198.859400,2.644400,37.148100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.768649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790931,0,3562.2303,52.203125,32.046875,815.453875,1,2,1175.660175,15.350675,52.728425,355.015300,7.125000,15.062500,0.000000,0.000000,0.000000,0.262695
1790932,4,2898.6813,23.053571,13.199986,273.049857,4,3,296.410943,6.052186,25.126200,3.782143,0.233057,1.326471,0.000000,0.000000,0.000000,0.640358
1790933,8,1732.8873,83.057750,4.540683,288.814567,1,16,331.347183,2.041967,83.173917,10.515283,0.609483,2.738883,0.000000,0.000000,0.000000,0.617906
1790934,8,7208.6328,43.584450,40.175350,693.526650,5,0,1308.887900,15.457300,51.257850,504.934150,7.507200,22.396550,61.498750,1.939500,9.697650,0.509921


In [21]:
# Separate data by strategy
strategy_groups = joined_nest_partsavg_df.groupby('fStrategies')

In [22]:
for strategy, group in strategy_groups:
    print(str(strategy)+" "+str(group.shape))

-2147483648 (520786, 17)
0 (692368, 17)
1 (48044, 17)
2 (10651, 17)
4 (14756, 17)
8 (218856, 17)
16 (20268, 17)
32 (5968, 17)
64 (4433, 17)
128 (16858, 17)
256 (2465, 17)
512 (26634, 17)
1024 (12292, 17)
2048 (6585, 17)
4096 (13020, 17)
8192 (83085, 17)
16384 (93867, 17)


In [14]:
# Placeholder for models and scalers
models = {}
scalers = {}


In [16]:
# Function to create an ANN model
def create_ann(input_dim, learning_rate=0.001, dropout_rate=0.3, neurons=[128, 64]):
    model = Sequential()
    model.add(Dense(neurons[0], activation='relu', input_shape=(input_dim,)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_absolute_error')
    return model

In [15]:
# Placeholder for models and scalers
models = {}
scalers = {}


In [39]:

for strategy, group in strategy_groups:
    X = group.drop(columns=['CropUtil', 'fStrategies'])
    y = group['CropUtil']

    # Standardize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Train final model with best parameters
    input_dim = X.shape[1]
    model = create_ann(input_dim, learning_rate=0.001, dropout_rate=0.2, neurons=[128, 64])

    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Train the model
    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr], verbose=1)

    # Evaluate the model
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Strategy {strategy} - Final ANN MAE: {mae}')

    # Store the models and scalers
    models[strategy] = model
    scalers[strategy] = scaler

    

Epoch 1/20


c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 2 is smaller than n_iter=9. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10416/10416 ━━━━━━━━━━━━━━━━━━━━ 9s 780us/step - loss: 0.1658 - val_loss: 0.1265 - learning_rate: 0.0010
Epoch 2/20
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 8s 786us/step - loss: 0.1310 - val_loss: 0.1220 - learning_rate: 0.0010
Epoch 3/20
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 8s 793us/step - loss: 0.1274 - val_loss: 0.1194 - learning_rate: 0.0010
Epoch 4/20
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 8s 769us/step - loss: 0.1250 - val_loss: 0.1197 - learning_rate: 0.0010
Epoch 5/20
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 8s 782us/step - loss: 0.1237 - val_loss: 0.1178 - learning_rate: 0.0010
Epoch 6/20
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 8s 795us/step - loss: 0.1231 - val_loss: 0.1164 - learning_rate: 0.0010
Epoch 7/20
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 8s 774us/step - loss: 0.1225 - val_loss: 0.1172 - learning_rate: 0.0010
Epoch 8/20
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 8s 769us/step - loss: 0.1219 - val_loss: 0.1176 - learning_rate: 0.0010
Epoch 9/20
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 8s 754us/step - loss: 0.1216 - val_loss

c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 2 is smaller than n_iter=9. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13848/13848 ━━━━━━━━━━━━━━━━━━━━ 11s 778us/step - loss: 0.1508 - val_loss: 0.1204 - learning_rate: 0.0010
Epoch 2/20
13848/13848 ━━━━━━━━━━━━━━━━━━━━ 11s 788us/step - loss: 0.1228 - val_loss: 0.1134 - learning_rate: 0.0010
Epoch 3/20
13848/13848 ━━━━━━━━━━━━━━━━━━━━ 11s 794us/step - loss: 0.1189 - val_loss: 0.1110 - learning_rate: 0.0010
Epoch 4/20
13848/13848 ━━━━━━━━━━━━━━━━━━━━ 11s 792us/step - loss: 0.1168 - val_loss: 0.1085 - learning_rate: 0.0010
Epoch 5/20
13848/13848 ━━━━━━━━━━━━━━━━━━━━ 11s 783us/step - loss: 0.1158 - val_loss: 0.1082 - learning_rate: 0.0010
Epoch 6/20
13848/13848 ━━━━━━━━━━━━━━━━━━━━ 11s 787us/step - loss: 0.1150 - val_loss: 0.1093 - learning_rate: 0.0010
Epoch 7/20
13848/13848 ━━━━━━━━━━━━━━━━━━━━ 11s 778us/step - loss: 0.1147 - val_loss: 0.1074 - learning_rate: 0.0010
Epoch 8/20
13848/13848 ━━━━━━━━━━━━━━━━━━━━ 11s 757us/step - loss: 0.1135 - val_loss: 0.1060 - learning_rate: 0.0010
Epoch 9/20
13848/13848 ━━━━━━━━━━━━━━━━━━━━ 11s 758us/step - loss: 0.1135 -

...

261/261 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3004 - val_loss: 0.1161 - learning_rate: 0.0010
Epoch 2/20
261/261 ━━━━━━━━━━━━━━━━━━━━ 0s 919us/step - loss: 0.1459 - val_loss: 0.1054 - learning_rate: 0.0010
Epoch 3/20
261/261 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - loss: 0.1272 - val_loss: 0.1008 - learning_rate: 0.0010
Epoch 4/20
261/261 ━━━━━━━━━━━━━━━━━━━━ 0s 902us/step - loss: 0.1182 - val_loss: 0.0973 - learning_rate: 0.0010
Epoch 5/20
261/261 ━━━━━━━━━━━━━━━━━━━━ 0s 898us/step - loss: 0.1110 - val_loss: 0.0871 - learning_rate: 0.0010
Epoch 6/20
261/261 ━━━━━━━━━━━━━━━━━━━━ 0s 876us/step - loss: 0.1046 - val_loss: 0.0850 - learning_rate: 0.0010
Epoch 7/20
261/261 ━━━━━━━━━━━━━━━━━━━━ 0s 887us/step - loss: 0.0997 - val_loss: 0.0820 - learning_rate: 0.0010
Epoch 8/20
261/261 ━━━━━━━━━━━━━━━━━━━━ 0s 898us/step - loss: 0.0971 - val_loss: 0.0865 - learning_rate: 0.0010
Epoch 9/20
261/261 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step - loss: 0.0950 - val_loss: 0.0801 - learning_rate: 0.0010
Epoch

c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 2 is smaller than n_iter=9. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1662/1662 ━━━━━━━━━━━━━━━━━━━━ 2s 830us/step - loss: 0.1738 - val_loss: 0.0992 - learning_rate: 0.0010
Epoch 2/20
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 1s 799us/step - loss: 0.1035 - val_loss: 0.0931 - learning_rate: 0.0010
Epoch 3/20
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 1s 798us/step - loss: 0.0948 - val_loss: 0.0858 - learning_rate: 0.0010
Epoch 4/20
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 1s 805us/step - loss: 0.0909 - val_loss: 0.0838 - learning_rate: 0.0010
Epoch 5/20
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 1s 791us/step - loss: 0.0883 - val_loss: 0.0834 - learning_rate: 0.0010
Epoch 6/20
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 1s 803us/step - loss: 0.0873 - val_loss: 0.0799 - learning_rate: 0.0010
Epoch 7/20
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 1s 783us/step - loss: 0.0871 - val_loss: 0.0787 - learning_rate: 0.0010
Epoch 8/20
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 1s 788us/step - loss: 0.0851 - val_loss: 0.0797 - learning_rate: 0.0010
Epoch 9/20
1662/1662 ━━━━━━━━━━━━━━━━━━━━ 1s 791us/step - loss: 0.0844 - val_loss: 0.0808 - learnin

c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 2 is smaller than n_iter=9. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\cslabwin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1878/1878 ━━━━━━━━━━━━━━━━━━━━ 2s 837us/step - loss: 0.1788 - val_loss: 0.0896 - learning_rate: 0.0010
Epoch 2/20
1878/1878 ━━━━━━━━━━━━━━━━━━━━ 1s 783us/step - loss: 0.0942 - val_loss: 0.0852 - learning_rate: 0.0010
Epoch 3/20
1878/1878 ━━━━━━━━━━━━━━━━━━━━ 2s 798us/step - loss: 0.0883 - val_loss: 0.0875 - learning_rate: 0.0010
Epoch 4/20
1878/1878 ━━━━━━━━━━━━━━━━━━━━ 2s 800us/step - loss: 0.0859 - val_loss: 0.0821 - learning_rate: 0.0010
Epoch 5/20
1878/1878 ━━━━━━━━━━━━━━━━━━━━ 1s 779us/step - loss: 0.0843 - val_loss: 0.0805 - learning_rate: 0.0010
Epoch 6/20
1878/1878 ━━━━━━━━━━━━━━━━━━━━ 2s 792us/step - loss: 0.0843 - val_loss: 0.0803 - learning_rate: 0.0010
Epoch 7/20
1878/1878 ━━━━━━━━━━━━━━━━━━━━ 2s 789us/step - loss: 0.0829 - val_loss: 0.0806 - learning_rate: 0.0010
Epoch 8/20
1878/1878 ━━━━━━━━━━━━━━━━━━━━ 2s 791us/step - loss: 0.0821 - val_loss: 0.0784 - learning_rate: 0.0010
Epoch 9/20
1878/1878 ━━━━━━━━━━━━━━━━━━━━ 2s 791us/step - loss: 0.0821 - val_loss: 0.0779 - learnin

In [41]:
import os
import joblib
from keras.models import load_model

# Create directories if they don't exist
os.makedirs('models', exist_ok=True)
os.makedirs('scalers', exist_ok=True)

In [ ]:
# Save the models, scalers, and encoders
for strategy, group in strategy_groups:
    # Save the regression model
    model_path = f'models/{strategy}_rf_model.pkl'
    joblib.dump(models[strategy], model_path)
    
    # Save the scaler
    scaler_path = f'scalers/{strategy}_scaler.pkl'
    joblib.dump(scalers[strategy], scaler_path)
    

**Load the Saved models and Check MAE and MSE for Test Data**

In [23]:
import joblib
from sklearn.metrics import mean_squared_error

In [25]:

for strategy, group in strategy_groups:
    X = group.drop(columns=['CropUtil', 'fStrategies'])
    y = group['CropUtil']

    # Load the scaler
    scaler_path = f'scalers/{strategy}_scaler.pkl'
    scaler = joblib.load(scaler_path)

    model_path = f'models/{strategy}_rf_model.pkl'
    model = joblib.load(model_path)

    ##

    X_scaled = scaler.transform(X)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Evaluate the model
    y_pred = model.predict(X_test)
    mae = mean_squared_error(y_test, y_pred)
    print(f'Strategy {strategy} - ANN MAE: {mae}')


3255/3255 ━━━━━━━━━━━━━━━━━━━━ 1s 440us/step
Strategy -2147483648 - ANN MAE: 0.17141690402405493
4328/4328 ━━━━━━━━━━━━━━━━━━━━ 2s 449us/step
Strategy 0 - ANN MAE: 0.02514191891341782
301/301 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step
Strategy 1 - ANN MAE: 0.013655793369293621
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 849us/step
Strategy 2 - ANN MAE: 0.005907572402820202
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 663us/step
Strategy 4 - ANN MAE: 0.015734189601127126
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 487us/step
Strategy 8 - ANN MAE: 0.010604672261677443
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 627us/step
Strategy 16 - ANN MAE: 0.01066016896679092
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 919us/step
Strategy 32 - ANN MAE: 0.01514540548180336
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Strategy 64 - ANN MAE: 0.01600299332727579
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 648us/step
Strategy 128 - ANN MAE: 0.009791450454960068
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Strategy 256 - ANN MAE: 0.012629927302528729
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 566us/step
Strat